In [7]:
%%capture
!pip install stanza

In [9]:
%%capture
import stanza
stanza.download('es')
nlp = stanza.Pipeline('es')

2022-02-13 21:34:40 INFO: Downloading default packages for language: es (Spanish)...
2022-02-13 21:34:41 INFO: File exists: /root/stanza_resources/es/default.zip.
2022-02-13 21:34:47 INFO: Finished downloading models and saved to /root/stanza_resources.
2022-02-13 21:34:47 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |
| ner       | conll02 |

2022-02-13 21:34:47 INFO: Use device: cpu
2022-02-13 21:34:47 INFO: Loading: tokenize
2022-02-13 21:34:47 INFO: Loading: mwt
2022-02-13 21:34:47 INFO: Loading: pos
2022-02-13 21:34:48 INFO: Loading: lemma
2022-02-13 21:34:48 INFO: Loading: depparse
2022-02-13 21:34:48 INFO: Loading: ner
2022-02-13 21:34:49 INFO: Done loading processors!


In [6]:
file1 = open('dic-judeo-español.txt', 'r', encoding="utf-8")
Lines = file1.readlines()
dic=[]
for l in Lines:
  p = {"src":l.split(";")[0],"target":l.split(";")[1]}
  dic.append(p)

In [15]:
import re, string

def normalize(s):
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
    )
    for a, b in replacements:
        s = s.replace(a, b).replace(a.upper(), b.upper())
    return s

In [16]:
def remove_punctuation (s):
  return re.sub('[%s]' % re.escape(string.punctuation), ' ', s)

In [85]:
def judeo_parse(s):
  if s.find("h") != -1:
    indices = [i for i, x in enumerate(s) if x == "h"]
    flag_h = 0
    for h in indices:
      string_list = list(s)
      if h != 0:
        if s[h-2] != "c":
          string_list[h] = ""
      else:
        string_list[0] = ""
    s = "".join(string_list)
  if s.find("y") != -1:
    if len(s) == 1:
      s = s.replace("y","i")
  if s.find("ca") != -1:
    s = s.replace("ca","ka")
  if s.find("co") != -1:
    s = s.replace("co","ko")
  if s.find("ñ") != -1:
    s = s.replace("ñ","ny")
  if s.find("qu") != -1:
    s = s.replace("qu","k")
  if s.find("rd") != -1:
    s = s.replace("rd","dr")
  if s.find("cr") != -1:
    s = s.replace("cr","kr")
  if s.find("ll") != -1:
    s = s.replace("ll","y")
  if s.find("g") != -1:
    s = s.replace("g","j")
  return s
judeo_parse("hoy")

'oy'

In [103]:
phrase="estuvieron comprando un perro"

In [105]:
doc = nlp(phrase)
jud_phrase = ""
for sent in doc.sentences:
  for word in sent.words:
    flag1 = 0
    palabra = word.text
    palabra = palabra.lower()
    palabra = normalize(palabra)
    palabra = remove_punctuation(palabra)
    if word.upos == "AUX":
      print(word)
      palabra = "yo "+ palabra
    for d in dic:
      if palabra == d["src"]:
        palabra = d["target"]
        flag1 = 1
        break
    if flag1 == 0:
        palabra = judeo_parse(palabra)
    jud_phrase += palabra.replace("\n","") + " "
print(jud_phrase)

{
  "id": 1,
  "text": "estuvieron",
  "lemma": "estar",
  "upos": "AUX",
  "feats": "Mood=Ind|Number=Plur|Person=3|Tense=Past|VerbForm=Fin",
  "head": 2,
  "deprel": "aux",
  "start_char": 0,
  "end_char": 10
}
yo estuvieron komprando un perro 
